# Advanced Topics

## Benefit of Cross-Fitting in `compress` Module

In [1]:
import pandas as pd
from TabuLLM.embed import TextColumnTransformer

import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=openai_api_key)
google_project_id = os.getenv('VERTEXAI_PROJECT')
google_location = os.getenv('VERTEXAI_LOCATION')

openai_args = {
    'client': client
}
google_args = {
    'project_id': google_project_id,
    'location': google_location
}

df = pd.read_csv('../data/raw.csv')

#textCol = 'diagnoses'
textCol = 'operations'

embeddings = TextColumnTransformer(
    #model_type = 'st'
    #model_type = 'openai', openai_args = openai_args
    model_type = 'google', google_args = google_args
).fit_transform(df.loc[:, [textCol]])
print(f'Shape of embeddings: {embeddings.shape}')

features_baseline = ['is_female', 'age', 'height', 'weight', 'optime']
features_embedding = [f'X_{i}' for i in range(embeddings.shape[1])]
X = pd.concat([embeddings, df[features_baseline]], axis = 1)
y = df['aki_severity']

c:\Users\alire\anaconda3\envs\devTEFE\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Shape of embeddings: (830, 768)


In [2]:
from TabuLLM.compress import CompressClassifier
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

ct_crossfit = ColumnTransformer([
    ('baseline', 'passthrough', features_baseline)
    , ('embedding', CompressClassifier(n_neighbors = 50, ncv=10), features_embedding)
], remainder = 'drop')
pipeline_crossfit = Pipeline([
    ('coltrans', ct_crossfit)
    , ('logit', LogisticRegression(penalty = None, solver = 'newton-cholesky', max_iter = 1000))
])

ct_naive = ColumnTransformer([
    ('baseline', 'passthrough', features_baseline)
    , ('embedding', CompressClassifier(n_neighbors = 50, ncv=0), features_embedding)
], remainder = 'drop')
pipeline_naive = Pipeline([
    ('coltrans', ct_naive)
    , ('logit', LogisticRegression(penalty = None, solver = 'newton-cholesky', max_iter = 1000))
])

from sklearn.model_selection import KFold, cross_val_score, RepeatedKFold

#kf = KFold(n_splits = 50, shuffle = True, random_state = 4321)
kf = RepeatedKFold(n_splits = 5, n_repeats = 50, random_state = 1111)

auc_naive = cross_val_score(
    pipeline_naive
    , X, y, cv = kf
    , scoring = 'roc_auc'
    , n_jobs=10
)
auc_crossfit = cross_val_score(
    pipeline_crossfit
    , X, y, cv = kf
    , scoring = 'roc_auc'
    , n_jobs=10
)
print(f'Naive/Cross-fit AUC: {auc_naive.mean():.3f}/{auc_crossfit.mean():.3f}')

# test the correlation between the two sets of AUCs
from scipy.stats import pearsonr
corrtest = pearsonr(auc_naive, auc_crossfit)
print(f'Pearson correlation (p-value): {corrtest[0]:.3f} ({corrtest[1]:.3e})')

# paired t-test
from scipy.stats import ttest_rel
ttest = ttest_rel(auc_naive, auc_crossfit)
print(f'P-value of paired t-test: {ttest[1]:.3e}')

Naive/Cross-fit AUC: 0.643/0.681
Pearson correlation (p-value): 0.739 (1.767e-44)
P-value of paired t-test: 6.320e-56


## KNN+Cross-fit vs. Clustering

In [3]:
from TabuLLM.cluster import SphericalKMeans

skmeans = SphericalKMeans(n_clusters=10, n_init=100).fit(embeddings)
clusters_soft = skmeans.transform(embeddings)
clusters_hard = skmeans.predict(embeddings)
print(f'Shape of clusters_soft: {clusters_soft.shape}')
print(f'Shape of clusters_hard: {clusters_hard.shape}')

Shape of clusters_soft: (830, 10)
Shape of clusters_hard: (830,)


In [4]:
varnames_cluster_soft = ['X' + str(n) for n in range(clusters_soft.shape[1])]
varnames_baseline = ['is_female', 'age', 'height', 'weight', 'optime']
dfCluster_soft = pd.DataFrame(clusters_soft, columns=varnames_cluster_soft)
dfCombined_soft = pd.concat([df, dfCluster_soft], axis=1)
X_soft = dfCombined_soft[varnames_baseline + varnames_cluster_soft]

dfCluster_hard = pd.DataFrame(clusters_hard, columns=['cluster'])
dfCombined_hard = pd.concat([df, dfCluster_hard], axis=1)
X_hard = dfCombined_hard[varnames_baseline + ['cluster']]

In [5]:
from sklearn.preprocessing import OneHotEncoder

pipeline_soft = LogisticRegression(max_iter=10, solver = 'newton-cholesky', penalty=None)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', varnames_baseline),
        ('cat', OneHotEncoder(drop = 'first'), ['cluster'])
    ]
    , remainder='drop'
)
pipeline_hard = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression(max_iter=10, solver = 'newton-cholesky', penalty=None))])

In [6]:
auc_soft = cross_val_score(
    pipeline_soft
    , X_soft, y, cv = kf
    , scoring = 'roc_auc'
    , n_jobs=10
)
auc_hard = cross_val_score(
    pipeline_hard
    , X_hard, y, cv = kf
    , scoring = 'roc_auc'
    , n_jobs=10
)
print(f'Soft/Hard AUC: {auc_soft.mean():.3f}/{auc_hard.mean():.3f}')

Soft/Hard AUC: 0.686/0.692


In [7]:
#pearsonr(auc_crossfit, auc_hard)